# Test monthly average

First make up some data and calculate the averages, then try using my function, and see you get the same answer.

In [1]:
#import my functions
import sys
sys.path.append(sys.path[0]+'/../..')


import helpers.fileHandler as fh
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils._indexDefinitions as _index
from utils.timePeriod import averageForTimePeriod

In [2]:
import xarray
import numpy
import cftime
import pandas

I am going to make up some data, and put it in a pandas array first

In [3]:
#first make up some time period
dateRange=xarray.cftime_range(
    start=cftime.DatetimeNoLeap(1995,1,1),
    end=cftime.DatetimeNoLeap(1999,12,1),
    freq='M')

In [4]:
#put it into a pd Dataframe
testPd=pandas.DataFrame(index=dateRange)

testPd.index.name='time'

In [5]:
#add some rando data
for keys in _index.monthsOfInterest:
    testPd[keys]=5*numpy.random.rand(len(dateRange),1)

I am now going to calculate the averages in a really easy to understand way

In [6]:
#years of interest
expectedAv=pandas.DataFrame(index=[1995,1996,1997,1998])
expectedAv.index.name='year'

#for every index defined in the index file
for keys in _index.monthsOfInterest:
    answer = list()
    
    #grab the time period defined in the index file
    #this line is a bit lazy, i really should define these indeces some other way incase of systematic errors
    monthPeriod=_index.monthsOfInterest[keys].copy()

    #select the data for this index
    indexArray = testPd[keys]
    
    temp = 0
    #for every month in the data
    for monthIter in numpy.arange(0,len(indexArray),1):
        # a month in the time period of interest:
        if ((monthIter+1)>=monthPeriod[0] and (monthIter+1)<=monthPeriod[1]):
            temp += indexArray.values[monthIter]
        else:
        # not a month in the time period of interest
            if temp!=0:
                answer.append(temp/(monthPeriod[1]-monthPeriod[0]+1))
                temp = 0
                monthPeriod[0]+=12
                monthPeriod[1]+=12
    #save output for this index and move to the next one
    expectedAv[keys]=answer


expectedAv.round(5)

,nino34NoDetrend,nino4NoDetrend,eastIONoDetrend,dmi,nino34,nino4,eastIO,sam,lat1,lat2
year,,,,,,,,,,
1995,2.67903,2.78903,2.09322,2.49260,2.74536,2.61926,1.44644,2.45849,2.87079,1.94186
1996,2.52162,2.49579,1.35486,2.77909,2.40330,2.75574,3.02332,2.94438,2.26741,3.18314
1997,2.47268,3.07388,2.31327,1.99342,2.13736,2.29690,2.91804,2.81016,1.77632,2.85867
1998,2.18903,2.54924,3.08623,2.78053,2.15775,2.56820,2.03922,2.11259,3.17151,2.29419


Calculate it using the function

In [7]:
#first cast it into an xarray
testXr = xarray.Dataset(testPd)

# do the calculation
calculatedAvs=averageForTimePeriod(testXr)

calculatedAvs.round(5)

<xarray.Dataset>
Dimensions:          (year: 4)
Coordinates:
  * year             (year) int64 1995 1996 1997 1998
Data variables:
    nino34           (year) float64 2.745 2.403 2.137 2.158
    sam              (year) float64 2.458 2.944 2.81 2.113
    nino4NoDetrend   (year) float64 2.789 2.496 3.074 2.549
    dmi              (year) float64 2.493 2.779 1.993 2.781
    eastIO           (year) float64 1.446 3.023 2.918 2.039
    eastIONoDetrend  (year) float64 2.093 1.355 2.313 3.086
    lat2             (year) float64 1.942 3.183 2.859 2.294
    lat1             (year) float64 2.871 2.267 1.776 3.172
    nino34NoDetrend  (year) float64 2.679 2.522 2.473 2.189
    nino4            (year) float64 2.619 2.756 2.297 2.568

In [8]:
numpy.isclose(calculatedAvs.sam, expectedAv.sam)

array([ True,  True,  True,  True])

In [9]:
numpy.unique(testXr.time.dt.year)

array([1995, 1996, 1997, 1998, 1999])